In [3]:
from scitiff.data import hyperstack_example
import scipp as sc
import numpy as np
import os
from PIL import Image

%matplotlib widget
import plopp as pp
import scipp as sc

In [4]:

sample_img = hyperstack_example()
sample_img

<scipp.DataArray>
Dimensions: Sizes[c:3, t:4, z:2, y:300, x:400, ]
Coordinates:
* c                          string        <no unit>  (c)  ["R", "G", "B"]
* t                           int64              [s]  (t)  [0, 1, 2, 3]
* x                         float64             [mm]  (x)  [0, 1.00251, ..., 398.997, 400]
* y                         float64             [mm]  (y)  [0, 1.00334, ..., 298.997, 300]
* z                           int64             [mm]  (z)  [10, 20]
Data:
                            float32         [counts]  (c, t, z, y, x)  [0, 1, ..., 89998, 89999]

In [5]:
tiff_data_file = "../data/tiff/SmallTurbine_0001.tiff"
assert os.path.exists(tiff_data_file), f"File not found: {tiff_data_file}"

In [6]:
# Load the TIFF file using PILLOW
data = np.array(Image.open(tiff_data_file))

In [7]:
# interactive plots

In [10]:
from plopp.widgets.drawing import DrawingTool
from functools import partial
from mpltoolbox import Rectangles


def define_mask(da, rect_info):
    """
    Function that creates a mask inside the area
    covered by the rectangle.
    """
    x = rect_info['x']
    y = rect_info['y']
    b = min(y['bottom'], y['top'])
    t = max(y['bottom'], y['top'])
    l = min(x['left'], x['right'])
    r = max(x['left'], x['right'])

    xcoord = sc.midpoints(da.coords[x['dim']])
    ycoord = sc.midpoints(da.coords[y['dim']])
    return (xcoord >= l) & (xcoord <= r) & (ycoord >= b) & (ycoord <= t)


def _get_rect_info(artist, figure):
    """
    Convert the raw rectangle info to a dict containing the dimensions of
    each axis, and values with units.
    """
    return lambda: {
        'x': {
            'dim': figure.canvas.dims['x'],
            'left': sc.scalar(artist.xy[0], unit=figure.canvas.units['x']),
            'right': sc.scalar(
                artist.xy[0] + artist.width, unit=figure.canvas.units['x']
            ),
        },
        'y': {
            'dim': figure.canvas.dims['y'],
            'bottom': sc.scalar(artist.xy[1], unit=figure.canvas.units['y']),
            'top': sc.scalar(
                artist.xy[1] + artist.height, unit=figure.canvas.units['y']
            ),
        },
    }


RectangleTool = partial(
    DrawingTool, tool=Rectangles, get_artist_info=_get_rect_info, icon='vector-square'
)

In [12]:
data_node = pp.Node(data)

def apply_masks(data, *masks):
    out = data.copy()
    for i, mask in enumerate(masks):
        out.masks[str(i)] = mask
    return out


masking_node = pp.Node(apply_masks, data_node)

fig = pp.imagefigure(masking_node, logc=True)

r = RectangleTool(
    figure=fig, input_node=data_node, func=define_mask, destination=masking_node
)
fig.toolbar['roi'] = r



AttributeError: 'numpy.ndarray' object has no attribute 'dims'